# Project Dreamer

#### Timeline: Sep 2021 - Nov 2021 @ HP
#### Objective: To create automation tools for a new imported business model and generate business insights for material planusers

### This python script is to map KB raw data sources with maintained databse and run in KB_Demand_WF.xlsm

In [ ]:
'''
-version 2.0-
add line 46 to convert string

-version 3.0-
add line 89 to change to df.pivot_table

-version 4.0-
add line 88to102 to fillna with description to solve index not shown issue with pandas pivot table

-version 5.0-
adjust mapping logic due to Cook go-live and Lisa's feedback
'''

import pandas as pd
import sys
pd.options.mode.chained_assignment = None

whole_path = sys.argv[0]
# Find folder of current file
loc = whole_path.rfind("\\")
path = whole_path[0:loc]

# NRP OPT WF Raw data
raw = pd.read_csv(path + '\Data source\Raw data.csv')

# Material Description
desc = pd.read_excel(path+ '\Keyboard Material Description.xlsx', sheet_name='Sheet1')

# Master Data

color = pd.read_excel(path+ '\Keyboard Solution Master Data.xlsx', sheet_name='SA to Map')
local = pd.read_excel(path+ '\Keyboard Solution Master Data.xlsx', sheet_name='Localization')
exception = pd.read_excel(path+ '\Keyboard Solution Master Data.xlsx', sheet_name='Exceptional TAM')
tam = pd.read_excel(path+ '\Keyboard Solution Master Data.xlsx', sheet_name='TAM')



# Clean data in raw
raw['Reference Date'] = pd.to_datetime(raw['Reference Date'], format='%m/%d/%Y')
raw['Week'] = pd.to_datetime(raw['Week'], format='%m/%d/%Y')

grouped = raw.groupby(["Reference Date","Location","Region","Material Number","CP Family Code",\
                       "Week","CP Business","CP Brand name","CP Segment"])["NRP"].sum().reset_index()
# Replace "#" with "-"
grouped['Material Number'] = grouped['Material Number'].str.replace('#','-')


# Bring in material description based on material number
wf = grouped.merge(desc[['Material Number','Material Description']], on = "Material Number",how="left")

# Spliting material number by -
wf[['SA# 6','SA# 3']] = wf['Material Number'].str.split(pat = '-' , expand=True)

# strip and upper for every object column in master data
for df in [color,local,exception,tam]:
    cols = df.select_dtypes(object).columns
    df[cols] = df[cols].fillna('')
    df[cols] = df[cols].astype(str)
    df[cols] = df[cols].apply(lambda x: x.str.strip())
    df[cols] = df[cols].apply(lambda x: x.str.upper())


# left join type & color
wf_color = wf.merge(color[['ODM','PLATFORM','SIZE','LAYOUT GROUP','REPORT GROUP','KEYBOARD SEGMENT',\
                         'FAMILY PLATFORM','HP KEYBOARD NO.','SA DASH CODE','KB TYPE','KB COLOR']], how="left",left_on = 'SA# 6', right_on = 'SA DASH CODE')
wf_color.drop(columns = ['PLATFORM','SA DASH CODE'],inplace= True)

# left join localization
wf_local = wf_color.merge(local[['SA DASH CODE','LAYOUT GROUP','LANGUAGE-EN','KEYBOARD LAYOUT']], \
                          how="left",left_on = ['SA# 3','LAYOUT GROUP'], right_on = ['SA DASH CODE','LAYOUT GROUP'])
wf_local.drop(columns = ['SA DASH CODE'],inplace= True)

# Remove accessory from KB Type
remove_accessory = wf_local[wf_local['KB TYPE']!='ACCESSORY']

# Reorder columns
reorder = remove_accessory[["Region", "CP Segment","CP Brand name", "Location", "CP Business","CP Family Code",\
                            "Material Number",  "Material Description","SA# 6", "SA# 3", "ODM", "SIZE", "LAYOUT GROUP", \
                            "REPORT GROUP", "KEYBOARD SEGMENT", "FAMILY PLATFORM", "HP KEYBOARD NO.", "KB TYPE", \
                            "KB COLOR", "LANGUAGE-EN", "KEYBOARD LAYOUT","Reference Date", \
                            "Week",  "NRP"]]


reorder['Material Description'] = reorder['Material Description'].fillna('')
reorder['ODM'] = reorder['ODM'].fillna('')
reorder['SIZE'] = reorder['SIZE'].fillna('')
reorder['LAYOUT GROUP'] = reorder['LAYOUT GROUP'].fillna('')
reorder['REPORT GROUP'] = reorder['REPORT GROUP'].fillna('')
reorder['KEYBOARD SEGMENT'] = reorder['KEYBOARD SEGMENT'].fillna('')
reorder['FAMILY PLATFORM'] = reorder['FAMILY PLATFORM'].fillna('')
reorder['HP KEYBOARD NO.'] = reorder['HP KEYBOARD NO.'].fillna('')
reorder['KB TYPE'] = reorder['KB TYPE'].fillna('')
reorder['KB COLOR'] = reorder['KB COLOR'].fillna('')
reorder['LANGUAGE-EN'] = reorder['LANGUAGE-EN'].fillna('')
reorder['KEYBOARD LAYOUT'] = reorder['KEYBOARD LAYOUT'].fillna('')

# Change SIZE from float to str
reorder['SIZE'] = reorder['SIZE'].astype(str)

# Pivot week - NRP
pivot_list = ["Region", "CP Segment","CP Brand name", "Location", "CP Business","CP Family Code",\
                            "Material Number", "Material Description", "SA# 6", "SA# 3", "ODM", "SIZE", "LAYOUT GROUP", \
                            "REPORT GROUP", "KEYBOARD SEGMENT", "FAMILY PLATFORM", "HP KEYBOARD NO.", "KB TYPE", \
                            "KB COLOR", "LANGUAGE-EN", "KEYBOARD LAYOUT","Reference Date"]


pivoted = reorder.pivot_table(index = pivot_list,columns = 'Week', values = 'NRP')



# Export to csv
pivoted.to_csv(path+"\Data source\WF.csv")
exception.to_csv(path+"\Data source\exception.csv", index=False)
tam.to_csv(path+"\Data source\\tam.csv", index=False)